# Lab | Data cleaning and wrangling

For this lab, we will be using the same dataset we used in the previous labs. We recommend using the same notebook since you will be reusing the same variables you previous created and used in labs. 

### Instructions

So far we have worked on `EDA`. This lab will focus on data cleaning and wrangling from everything we noticed before.

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('we_fn_use_c_marketing_customer_value_analysis.csv')
data.set_index('Customer', drop=True, inplace=True)

#### 1. We will start with removing outliers. So far, we have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.

In [3]:
def remove_outliers(df, threshold=1.5):
    numerical = df.select_dtypes(np.number)
    columns = numerical.columns
    for column in columns:
        if len(df[column].unique()) < 100:
            continue
        else:
            upper = np.percentile(df[column], 75)
            lower = np.percentile(df[column], 25)
            iqr = upper - lower
            upper_limit = upper + threshold * iqr
            lower_limit = lower - threshold * iqr
            df = df[(df[column] > lower_limit) & (df[column] < upper_limit)]
    return df

In [4]:
data = remove_outliers(data)

#### 2. Create a copy of the dataframe for the data wrangling.

In [5]:
data_copy = data.copy()

#### 3. Normalize the continuous variables. You can use any one method you want.

In [6]:
def split_numericals(data, threshold=20):
    num = data.select_dtypes(np.number)
    cont_columns = []
    disc_columns = []
    for col in num.columns:
        if len(num[col].unique()) > threshold:
            cont_columns.append(col)
        else:
            disc_columns.append(col)
    continuous_df = data.loc[:, cont_columns]
    discrete_df = data.loc[:, disc_columns]
    return continuous_df, discrete_df

In [7]:
continuous_data, discrete_data = split_numericals(data_copy)

In [8]:
transformer = StandardScaler().fit(continuous_data)
x_standardized = transformer.transform(continuous_data)
x_standardized = pd.DataFrame(x_standardized, columns=continuous_data.columns)

In [9]:
x_standardized.head()

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Total Claim Amount
0,-1.065356,0.583423,-0.777085,1.697383,-1.542810,0.028144
1,2.184071,0.335939,1.020067,0.299696,-0.358507,0.935451
2,0.501708,-1.271769,0.927906,0.299696,0.610469,0.752698
3,-1.049252,0.173378,-0.592762,-0.299312,-0.143179,-1.203902
4,0.697636,0.801930,-0.777085,-0.099642,1.651220,-1.097758


#### 4. The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.

In [10]:
data['Effective To Date'] = pd.to_datetime(data['Effective To Date'])

In [11]:
data['Month'] = data['Effective To Date'].dt.month

In [12]:
data['Day Of Week'] = data['Effective To Date'].dt.dayofweek

In [13]:
data.drop('Effective To Date', axis=1, inplace=True)

#### 5. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

**Hint for Categorical Variables**

- You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):

```
# One hot to state
# Ordinal to coverage
# Ordinal to employmentstatus
# Ordinal to location code
# One hot to marital status
# One hot to policy type
# One hot to policy
# One hot to renew offercustomer_df
# One hot to sales channel
# One hot vehicle class
# Ordinal vehicle size

data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
# given that column "coverage" in the dataframe "data" has three categories:
# "basic", "extended", and "premium" and values are to be represented in the same order.
```

In [14]:
# Encode ordinal columns
ordinal_cols = ['Coverage', 'EmploymentStatus', 'Location Code', 'Vehicle Size']
ordinal = data_copy.loc[:, ordinal_cols]

for col in ordinal_cols:
    print(data[col].value_counts())

Basic       4923
Extended    2313
Premium      553
Name: Coverage, dtype: int64
Employed         4957
Unemployed       1859
Medical Leave     373
Disabled          349
Retired           251
Name: EmploymentStatus, dtype: int64
Suburban    4804
Rural       1574
Urban       1411
Name: Location Code, dtype: int64
Medsize    5484
Small      1492
Large       813
Name: Vehicle Size, dtype: int64


In [15]:
data["Coverage"] = data["Coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})

In [16]:
data["Vehicle Size"] = data["Vehicle Size"].map({"Small" : 0, "Medsize" : 1, "Large" : 2})

In [17]:
data["Location Code"] = data["Location Code"].map({"Urban" : 0, "Suburban" : 1, "Rural" : 2})

In [18]:
data["EmploymentStatus"] = data["EmploymentStatus"].map({"Urban" : 0, "Suburban" : 1, "Rural" : 2})

In [19]:
# Combine categorical & discrete numerical data
categoricals = data_copy.select_dtypes(object)
total_categoricals = pd.concat([categoricals, discrete_data], axis=1)

In [20]:
cat_encoded = pd.get_dummies(total_categoricals, columns=total_categoricals.columns, \
    drop_first=True)

In [21]:
# Save data for next lab
data.to_csv('clean_data.csv')